## Import packages

In [9]:
!pwd

/bin/sh: 1: cd: can't cd to Project


In [15]:
import Get_global_local_feature
import scipy.signal as ss
import pac
import itertools
import pickle
import mne
import os
import scipy
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
%matplotlib inline

# from numba import jit, cuda


## Get global feature using convolution

In [17]:
suffix = '_1_200'#'_1ch_nv'
gamma = [ 1, 200]
beta  = [ 1, 50]

filt_wind = np.ones((3, 3))
filt_wind /= filt_wind.sum()

with open('config.json') as f:
    config = json.load(f)
base_path = config['BASE_PATH']
ds_path = os.path.join(base_path, 'ds003490-download')
nbchan = len(config['channels']) - 1
nbtime = 601

base_shape = (3, 25, 3)
mvl_shape = base_shape + (nbchan, gamma[1] - gamma[0] + 1, beta[1] - beta[0] + 1)

In [18]:
mvl_2ds_mmap_path = os.path.join(ds_path, f'MVL_2ds_{suffix}.mmap')
mvl_2ds = np.memmap(mvl_2ds_mmap_path, dtype=float, mode='r',
                    shape=mvl_shape)

mvl_time_mmap_path = os.path.join(ds_path, f'PAC_time_{suffix}.mmap')
mvl_time = np.memmap(mvl_time_mmap_path, dtype=float, mode='r',
                    shape=mvl_shape + (6,))

erp_mmap_path = os.path.join(ds_path, f'ERPs_{suffix}.mmap')
epochs = np.memmap(erp_mmap_path, dtype=float, mode='r',
                    shape=base_shape + (2, nbchan, nbtime))

## Working on Kiani's PAC
### new pipline with normalization of channel
load PAC

In [ ]:
a = np.load('..\data\mvl_2ds_1_50.npz')
PAC = a['mvl_2ds']*1e-12
Test = Get_global_local_feature.conv_PAC_dist(
    PAC[0, 0, 0, 0, 0:64, :], h=4, w=4, down_sample_factor_h=4, down_sample_factor_w=4)
Test.shape


In [ ]:
PAC_feature_sub_on_electrodes = np.zeros(shape=(3, 25, 3, 63, 16, 4))


@jit
def PAC_caluculation():
    for i in np.arange(3):
        for j in np.arange(3):
            for ch in np.arange(63):
                for sub in np.arange(25):
                    PAC_feature_sub_on_electrodes[i, sub, j, ch, :, :] = Get_global_local_feature.conv_PAC_dist(
                        PAC[i, sub, j, ch, 0:64, :], h=4, w=4, down_sample_factor_h=4, down_sample_factor_w=4)


if __name__ == "__main__":
    PAC_caluculation()
    np.save(f'..\data\Feature_window_dimension_4_4.npy',
            PAC_feature_sub_on_electrodes)


### Load feature and set parameter

In [ ]:
PAC_feature_sub_on_electrodes = np.load(
    f'..\data\Feature_window_dimension_4_4.npy')
groups = ['PD Med Off', 'PD Med On', 'CTL']
event_types = ['Target', 'Standard', 'Novelty']
PAC_feature_sub_on_electrodes.shape


#### Get indexes of selected channels

In [ ]:
with open('..\data\channels_name.pkl', 'rb') as f:
    channels_name = pickle.load(f)
selected_channels = ['Fz', 'Pz', 'Cz', 'FCz']
# selected_channels = ['FCz']
list_index = [channels_name.index(i) for i in selected_channels]
list_index


## Get Local Features for time PAC series


In [ ]:
def conv_PAC_dist(PAC_dist, h=4, w=4, down_sample_factor_h=4, down_sample_factor_w=4, filter_mode='valid'):
    """PAC_dist input PAC values\
    h filter kernel size along axis 0\
    w filter kernel size along axis 1\
    down sample factor h along axis 0\
    down sample factor w along axis 1\
    filter mode define the mode of convolve2d of scipy.signal"""
    # Filtering
    width_filter = w
    height_filter = h
    kernel_filter = np.ones(
        shape=[width_filter, height_filter])/(width_filter*height_filter)
    PAC_feature = ss.convolve2d(PAC_dist, kernel_filter, mode=filter_mode)

    # Sampling
    height_index = np.linspace(
        0, PAC_feature.shape[0]-1, PAC_dist.shape[0]//down_sample_factor_h).astype(np.int32)
    width_index = np.linspace(
        0, PAC_feature.shape[0]-1, PAC_dist.shape[1]//down_sample_factor_w).astype(np.int32)
    h, w = np.meshgrid(height_index, width_index)
    return PAC_feature[w, h]


In [23]:
# Here load a matrix with shape of 3,25,3,63,num_High_fre,num_Low_fre,num_time_window=6 cotaining PAC values
Num_High_fre = gamma[1] - gamma[0] + 1
Num_Low_fre = beta[1] - beta[0] + 1
PAC_distribution_time_window = mvl_time


# here you can set the dimention of Kernel filter and save path for
Convolution_window_h = 4
Convolution_window_l = 4
Num_High_feature = Num_High_fre // Convolution_window_h
Num_Low_feature = Num_Low_fre // Convolution_window_l
Save_path = os.path.join(ds_path, 'PAC_time_conv_{suffix}.mmap')
PAC_feature_time_window = np.memmap(Save_path, dtype=float, shape=(
    3, 25, 3, 63, Num_High_feature, Num_Low_feature, 6), mode='w+')

# # Test
# PAC_feature_time_window[1,1,1,1,:,:,1] = Get_global_local_feature.conv_PAC_dist(PAC_distribution_time_window[1,1,1,1,:,:,1],
#                                          h=Convolution_window_h,w=Convolution_window_l,down_sample_factor_w=Convolution_window_l,down_sample_factor_h=Convolution_window_h)


PAC_feature_time_window = np.memmap('../data/Test.mmap', dtype=float, shape=(
    3, 25, 3, 63, Num_High_feature, Num_Low_feature, 6), mode='w+')
window_size = 200  # in ms
down_sample_factor = 1
window_sample = int(window_size*(0.5/down_sample_factor))
frequency_h_n = 76
frequency_l_n = 23
start_high_frequency = 25
end_high_frequency = 100
start_low_frequency = 8
end_low_frequency = 30
frequency_range_high = np.linspace(
    start_high_frequency, end_high_frequency, frequency_h_n).astype(np.int64)
frequency_range_low = np.linspace(
    start_low_frequency, end_low_frequency, frequency_l_n).astype(np.int64)

# attention !!!! attention !!!! attention !!!!
# If you run this cell for the second time or more, after possible crash in middle of the first run, you have to change the
# "load mode" of np.mmap from 'w+' to 'r+', ohterwise, all the saved calculation, unitll the middle of the first run
# are going to be removed by loading in 'w+' mode. After this change, there is no need to be worry of deleting previous
# calculation you have done in next runs.


PAC_feature_sub_on_electrodes = np.memmap('../data/Test.mmap', dtype=float, shape=(
    3, 25, 3, 63, frequency_h_n, frequency_l_n, 6), mode='r+')
# PAC_feature_sub_on_electrodes = np.zeros([3, 25, 3, 63,frequency_h_n ,frequency_l_n,6])
print(PAC_feature_sub_on_electrodes.flags)


def wrapper(i, sub, j, ch):
    for t in np.arange(6):
        PAC_feature_time_window[i, sub, j, ch, :, :, t] = \
            Get_global_local_feature.conv_PAC_dist(PAC_distribution_time_window[i, sub, j, ch, :, :, t],
                                                   h=Convolution_window_h, w=Convolution_window_l,
                                                   down_sample_factor_w=Convolution_window_l,
                                                   down_sample_factor_h=Convolution_window_h)


Parallel(n_jobs=12, verbose=10)(delayed(wrapper)(i, sub, j, ch) \
                                for i, sub, j, ch in itertools.product(range(3), range(25), range(3), range(nbchan)))


  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False



[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.8s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.9s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    1.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.1898s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    1.0s
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:    1.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0968s.) Setting batch_size=4.
[Parallel(n_jobs=12)]: Done  54 tasks      | elapsed:    1.2s
[Parallel(n_jobs=12)]: Done  75 tasks      | elapsed:    1.3s
[Parallel(n_jobs=12)]: Batch computation too fast (0.1991s.) Setting batch_size=8.
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:    1.5s
[Parallel(n_jobs=12)]: Done 164 tasks      | elapsed:    2.0s
[Parallel(n_jobs=12)]: Done 256 tasks      | elapsed:    2.9s
[Parallel(n_jobs=12)]: Done 392 tasks      | elapsed: 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

## P value of feature

In [ ]:
pv = np.zeros((PAC_feature_sub_on_electrodes.shape[3], 3, 3, 16, 4))
# for ch in range(PAC_feature_sub_on_electrodes.shape[3]):
for ch in list_index:
    #     print(ch)
    for grp, j in itertools.product(zip(itertools.combinations(range(3), 2), range(3)), range(3)):
        a, b = grp[0]
        i = grp[1]
        test_data1, test_data2 = PAC_feature_sub_on_electrodes[a, :, j, ch].copy(
        ), PAC_feature_sub_on_electrodes[b, :, j, ch].copy()
        # for sub in range(PAC_feature_sub_on_electrodes.shape[1]):
        #     test_data1[:, sub] = sig.convolve2d(test_data1[:, sub], filt_wind, boundary='symm', mode='same')
        #     test_data2[:, sub] = sig.convolve2d(test_data2[:, sub], filt_wind, boundary='symm', mode='same')

        t, p = scipy.stats.ttest_ind(test_data1, test_data2, equal_var=False)
        p[p > 0.05] = 1
        pv[ch, i, j] = (-np.log(p))

i = 1


In [ ]:
# x axis ticks formater
def format_func_x(value, tick_number):
    # find number of multiples of pi/2
    if tick_number % 2 == 1:
        return f'({1+4*(value)},{1+4*(value+1)-1})'
# y axis ticks formater


def format_func_y(value, tick_number):
    # find number of multiples of pi/2
    if tick_number % 2 == 1:
        return f'({10+4*(value)},{10+4*(value+1)-1})'


gamma = [1, 16]
beta = [1, 4]


def plot_pac(pac, high_freq=gamma, low_freq=beta, ax=None, **kwargs):
    if ax is None:
        fig = plt.figure(figsize=(7, 15))
        ax = fig.subplots()

    im = ax.imshow((pac), origin='lower', interpolation='nearest',  # 'nearest',
                   #                    aspect='auto', )
                   aspect=np.diff(low_freq)/np.diff(high_freq), cmap='RdBu_r', **kwargs)

    if ax is None:
        plt.show()

    return im


In [ ]:
Path_to_save_fig = 'D:\Mastersharif\MasterProject\Working_with_kiani\\articles\Result_on_articles\Cbar_Corrected'
temp = pv.sum(axis=0)
vmax = temp.max()
vmin = temp.min()
max_1 = 0
fig, axs = plt.subplots(3, 3, sharex=True, sharey=True, figsize=(15, 15))
#     vmin, vmax = get_percent(np.log(mvl_2ds[:, :, :, ch].mean(axis=1)), 0.83)
for i, j in itertools.product(range(3), range(3)):
    im = plot_pac(temp[i, j], ax=axs[i, j], vmin=vmin, vmax=vmax)
#                       vmin=vmin vmax=vmax)

    axs[i, j].xaxis.set_visible(False)
    axs[i, j].yaxis.set_visible(False)
    axs[i, j].xaxis.set_major_formatter(plt.FuncFormatter(format_func_x))
    axs[i, j].yaxis.set_major_formatter(plt.FuncFormatter(format_func_y))
cbar_ax = fig.add_axes([0.95, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax)
# cbar = fig.colorbar(ims[i][j], ax=axs.ra//vel().tolist(), shrink=0.95)

for grp in zip(itertools.combinations(range(3), 2), range(3)):
    a, b = grp[0]
    i = grp[1]

    axs[i, 0].set_ylabel(f'{groups[a]} vs {groups[b]}')
    axs[0, i].set_title(event_types[i])

    axs[-1, i].xaxis.set_visible(True)
    axs[i, 0].yaxis.set_visible(True)

fname = f'neq_var_p_value_dist_feature_h_feature_l_window_dimetion_4_RdBu_r_Selected_Chs_corrected_cbar'
file_path = os.path.join(Path_to_save_fig, fname)
plt.savefig(file_path)


In [ ]:
a = np.array([[3, 2, 5], [8, 1, 2], [6, 6, 7], [3, 5, 1]])
index = np.linspace(a.shape[0]-1, 0, a.shape[0]).astype(np.int32)
a[index, :]


## How imshow plot work

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

A = np.array([[3, 2, 5], [8, 1, 2], [6, 6, 7], [3, 5, 1]])  # The array to plot

im = plt.imshow(A, origin="lower", interpolation="nearest", cmap=plt.cm.gray_r)
plt.colorbar(im)


## Plot topography of important feature


In [ ]:
Path_to_save_fig = 'D:\Mastersharif\MasterProject\Working_with_kiani\\articles\Result_on_articles\\topography'

feature_h = 0
feature_l = 1
with open('..\data\channels_name.pkl', 'rb') as f:
    channels_name = pickle.load(f)
PAC_feature_sub_on_electrodes_gm = np.mean(
    PAC_feature_sub_on_electrodes[:, :, :, :, feature_h, feature_l], axis=1)
vmin = np.min(PAC_feature_sub_on_electrodes_gm)
vmax = np.max(PAC_feature_sub_on_electrodes_gm)
fig, axs = plt.subplots(3, 3, sharex=True, sharey=True, figsize=(15, 15))

mne_info = mne.create_info(ch_names=channels_name, sfreq=500., ch_types='eeg')

montage = mne.channels.read_custom_montage('Standard-10-20-Cap81.locs')

for i, j in itertools.product(range(3), range(3)):
    #     data = mvls[i, :, j, :].mean(axis=0).reshape((-1, 1))
    temp = np.expand_dims(PAC_feature_sub_on_electrodes_gm[i, j, :], axis=1)
    PAC_grand_mean = mne.EvokedArray(temp, mne_info)
    PAC_grand_mean.set_montage(montage)
    im, cm = mne.viz.plot_topomap(PAC_grand_mean.data[:, 0],
                                  PAC_grand_mean.info, axes=axs[i,
                                                                j], show=False,
                                  names=channels_name, show_names=True,
                                  vmin=vmin, vmax=vmax, res=200, cmap='RdBu_r', fontsize='medium')

for i in range(3):
    axs[i, 0].set_ylabel(groups[i])
    axs[0, i].set_title(event_types[i])
    axs[-1, i].xaxis.set_visible(True)
    axs[i, 0].yaxis.set_visible(True)


cbar_ax = fig.add_axes([0.95, 0.15, 0.02, 0.7])
clb = fig.colorbar(im, cax=cbar_ax)
fname = f'feature_h_{feature_h}_feature_l_{feature_l}_RdBu_r'
file_path = os.path.join(Path_to_save_fig, fname)
plt.savefig(file_path)


## P values on channel

In [ ]:
feature_h = 0
feature_l = 1
fig, axs = plt.subplots(3, 3, sharex=True, sharey=True, figsize=(15, 15))
# vmin, vmax = mvls.mean(axis=1).min(), mvls.mean(axis=1).max() #get_percent(mvls.mean(axis=1), 0.95)
for grp, j in itertools.product(zip(itertools.combinations(range(3), 2), range(3)), range(3)):
    a, b = grp[0]
    i = grp[1]
    data_a = PAC_feature_sub_on_electrodes[a, :, j, :, feature_h, feature_l]
    data_b = PAC_feature_sub_on_electrodes[b, :, j, :, feature_h, feature_l]
    t, p = scipy.stats.ttest_ind(data_a, data_b)
    p[p > 0.05] = 1
    p = p.reshape((-1, 1))
    mvl_evoked = mne.EvokedArray(-np.log(p),  # - -np.log(p).mean(),
                                 mne_info)
    mvl_evoked.set_montage(montage)
    im, cm = mne.viz.plot_topomap(mvl_evoked.data[:, 0],
                                  mvl_evoked.info, axes=axs[i, j],
                                  show=False,
                                  names=channels_name, show_names=True, cmap='RdBu_r')
    # , vmin=vmin- mvls.mean(), vmax=vmax- mvls.mean())

for grp in zip(itertools.combinations(range(3), 2), range(3)):
    a, b = grp[0]
    i = grp[1]

    axs[i, 0].set_ylabel(f'{groups[a]} vs {groups[b]}')
    axs[0, i].set_title(event_types[i])

    axs[-1, i].xaxis.set_visible(True)
    axs[i, 0].yaxis.set_visible(True)

cbar_ax = fig.add_axes([0.95, 0.15, 0.02, 0.7])
clb = fig.colorbar(im, cax=cbar_ax)
fname = f'p_value_feature_h_{feature_h}_feature_l_{feature_l}_RdBu_r'
file_path = os.path.join(Path_to_save_fig, fname)
plt.savefig(file_path)
